In [ ]:
# install dependencies
!pip install numpy matplotlib selenium tqdm scipy

In [ ]:
# config params to change when the belastingdienst page is changed

config = {
    'input_params': {
        'box1': {
            'eltotal': ['id', 'verdeeloptimaalinvoer_verdeeloptimaalinvoer1_invfmbx1ewtotafteigenwongez'],
            'elinput': ['id', 'verdeeloptimaalinvoer_verdeeloptimaalinvoer1_verdeelewaang'],
        },
        'box3': {
            'eltotal': ['id', 'verdeeloptimaalinvoer_verdeeloptimaalinvoer1_invfmbx3gezgrdslgez'],
            'elinput': ['id', 'verdeeloptimaalinvoer_verdeeloptimaalinvoer1_verdeelbox3aang'],
        },
    },
    'berekenbutton': ['xpath', '//button[@title="Belasting berekenen"]'],
    'resultfield': ['xpath', "//*[contains(text(), 'Te betalen')]"],
    'num_samples': 1_000,
}

In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt

from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

from tqdm.auto import tqdm

from scipy.interpolate import NearestNDInterpolator, RBFInterpolator
from scipy.stats import zscore
from scipy.optimize import minimize

import time

In [ ]:
# util functions

def toint(s):
    return int(s.replace('.', '').replace(',','.'))


def set_input(el, inp):
    el.send_keys(Keys.CONTROL + "a")
    el.send_keys(inp)
    el.send_keys(Keys.TAB)
    time.sleep(1)

In [ ]:
# initialize an instance of Firefox with selenium
# and wait for the user to go to the verdelen page

options = Options()
driver = Firefox(options=options)

driver.get('https://mijn.belastingdienst.nl/mbd-pmb/pmb.html')

is_aangifte_page = lambda driver: 'aangifte' in driver.current_url and '2022' in driver.current_url and 'olaib.html' in driver.current_url
    
while not is_aangifte_page(driver):
    time.sleep(1)
    
time.sleep(1)


while 'Verdeel de bedragen' not in driver.page_source:
    time.sleep(1)
    
time.sleep(1)


# retrieve the box1/box3 parameters that we need to optimize
enabled_params = []
totparams = {}
for k, v in config['input_params'].items():
    try:
        el = driver.find_element(*v['eltotal'])
        totparams[k] = toint(el.get_attribute('innerText'))
        enabled_params.append(k)
    except NoSuchElementException as e:
        print(f'WARN: Element not found: {k} {v["eltotal"]}')
    
print(f"Found the following params on page: {', '.join(enabled_params)}")

In [ ]:
num_params = len(enabled_params)
num_samples = config['num_samples']

samples = np.random.uniform(0, 1, (num_samples, num_params))

print('Distribution parameters')

for idx in range(samples.shape[1]):
    plt.figure(figsize=(10, 6))
    plt.hist(samples[:, idx], bins=10)
    plt.title(f'Histogram sampling distribution for {enabled_params[idx]}')
    plt.ylabel('Frequency')
    plt.xlabel('Parameter')

In [ ]:
def calc(samples):
    for param_idx, param in zip(range(len(enabled_params)), enabled_params):
        elinp = driver.find_element(*config['input_params'][param]['elinput'])
        perc = samples[param_idx]
        set_input(elinp, f'{int(totparams[param] * perc)}')
    
    driver.find_element(*config['berekenbutton']).click()
    time.sleep(1)

    betalen = toint(driver.find_element(*config['resultfield']).get_attribute('innerText').split('€')[-1].strip())
    return betalen


results = []
for idx in tqdm(range(samples.shape[0])):
    betalen = calc(samples[idx, :])
    results.append(betalen)

In [ ]:
# visualize the gradient based on the random samples

def visgrad(xy, z, interpolator, title, labels, interparams=None):
    if interparams is None:
        interparams = {}
    
    edges = np.linspace(0., 1., 101)
    centers = edges[:-1] + np.diff(edges[:2])[0] / 2.

    x_i, y_i = np.meshgrid(centers, centers)
    x_i = x_i.reshape(-1, 1)
    y_i = y_i.reshape(-1, 1)
    xy_i = np.concatenate([x_i, y_i], axis=1)


    interp = interpolator(xy, z, **interparams)
    z_i = interp(xy_i)

    # plot the result
    fig, ax = plt.subplots()

    X_edges, Y_edges = np.meshgrid(edges, edges)
    
    # Use red-white-blue colorpallet
    # Because they resemble the colors of the Dutch flag
    lims = dict(cmap='RdBu_r', vmin=z.min(), vmax=z.max())

    mapping = ax.pcolormesh(
        X_edges, Y_edges, z_i.reshape(100, 100),
        shading='flat', **lims
    )

    ax.scatter(xy[:, 0], xy[:, 1], 10, z, edgecolor='w', lw=0.1, **lims)
    ax.set(
        title=title,
        xlim=(0., 1.),
        ylim=(0., 1.),
    )
    
    plt.xlabel(labels[0])
    plt.ylabel(labels[1])

    cbar = fig.colorbar(mapping)
    cbar.ax.set_ylabel('Gradient (lower = better)', rotation=270)
    
    
# convert the amount to pay into zscore
# because I do not feel comfortable putting
# my box1/box3 numbers out in public
zresults = zscore(results)

# sort in descending order - this improves
# the matplotlib render because the lowest
# values are drawn last
sortidx = np.argsort(-np.array(zresults))

sorted_samples = samples.copy()[sortidx, :]
sorted_results = np.array(zresults)[sortidx]

# Visualize the found gradient using a NN and RBF Interpolator
visgrad(sorted_samples, sorted_results, NearestNDInterpolator, 'NN Interpolated Gradient', enabled_params)
visgrad(sorted_samples, sorted_results, RBFInterpolator, 'RBF Interpolated Gradient', enabled_params, {'epsilon': 2})

In [ ]:
topx = 10
topx_samples = sorted_samples[-topx:]

topx_samples

In [ ]:
# Run the scipy minimizer

method = 'Nelder-Mead'
options = {
    #'disp': True, 
    'return_all': True,
}

minimizer_results = []
bounds = [
    (0, 1) for i in range(2)
]

for sample in tqdm(topx_samples[::-1]):
    res = minimize(calc, np.array(sample), method=method, bounds=bounds, options=options)
    minimizer_results.append(res)

In [ ]:
success = [x for x in minimizer_results if x.success]
fun = [x.fun for x in success]
idxmin = fun.index(min(fun))
winning_sample = success[idxmin].x

for idx, param in zip(range(len(enabled_params)), enabled_params):
    perc = winning_sample[idx]
    
    left = int(totparams[param] * perc)
    right = int(totparams[param] - left)
    print(f'{param} Uw deel: {left} Partner: {right}')
print(f'Te betalen: €{int(fun[idxmin])}')
print(f'Verschil tussen min en max: €{max(results) - int(fun[idxmin])} max: {max(results)} min: {int(fun[idxmin])}')